# Installing Necessary Libraries


In [27]:
!pip install -q pandas openpyxl cvxpy numpy

In [28]:
!pip install ecos


# Import File

In [30]:
from google.colab import files
uploaded = files.upload()           # choose Candidate Optimizer Project.xlsx


Saving Candidate Optimizer Project.xlsx to Candidate Optimizer Project.xlsx


In [31]:
import pandas as pd

file_path = 'Candidate Optimizer Project.xlsx'

# ------------------------------------------------------------------
# 1. Asset-level data  ➜  df_assets
# ------------------------------------------------------------------
df_assets = pd.read_excel(file_path, sheet_name='Sample Data')
print(f"{len(df_assets)} assets loaded")

# ------------------------------------------------------------------
# 2. Credit-quality lookup  ➜  quality_map
# ------------------------------------------------------------------
df_credit = pd.read_excel(
    file_path,
    sheet_name='Data Key',
    skiprows=1,        # drop the big “Credit Quality / Liquidity” banner
    usecols='A:B'      # only the left-hand table
).dropna(how='all')    # remove any blank rows at the bottom

quality_map = dict(zip(df_credit['Credit Quality'], df_credit['Numeric']))

df_assets['Quality_Num'] = df_assets['Quality'].map(quality_map)

# ------------------------------------------------------------------
# 3. Liquidity lookup  ➜  liquidity_map
# ------------------------------------------------------------------
df_liq = pd.read_excel(
    file_path,
    sheet_name='Data Key',
    skiprows=1,
    usecols='D:E'
).dropna(how='all')

liquidity_map = dict(zip(df_liq['Liquidity Tier'], df_liq['Translation']))

df_assets['Liquidity_Label'] = df_assets['Liquidity Tier'].map(liquidity_map)

# ------------------------------------------------------------------
# Quick peek
# ------------------------------------------------------------------
display(df_assets.head())


24 assets loaded


,Asset,Asset Level Min Weight,Asset Level Max Weight,Sector,Yield,Duration,Quality,Liquidity Tier,Quality_Num,Liquidity_Label
0,6 Month Treasury,0.050,0.500,TSY,0.095,0.497,AAA,1,1,Same Day
1,1 Year Treasury,0.000,0.500,TSY,0.101,0.994,AAA,1,1,Same Day
2,2 Year Treasury,0.000,0.500,TSY,0.156,1.986,AAA,1,1,Same Day
3,3 Year Treasury,0.000,0.500,TSY,0.213,2.937,AAA,1,1,Same Day
4,5 Year Treasury,0.000,0.200,TSY,0.422,4.957,AAA,1,1,Same Day


In [32]:
# Drop duplicate rows
df_assets = df_assets.drop_duplicates()

In [33]:
# Shape
df_assets.shape

(24, 10)

In [34]:
# Print Df
display(df_assets)


,Asset,Asset Level Min Weight,Asset Level Max Weight,Sector,Yield,Duration,Quality,Liquidity Tier,Quality_Num,Liquidity_Label
0,6 Month Treasury,0.050,0.500,TSY,0.095,0.497,AAA,1,1,Same Day
1,1 Year Treasury,0.000,0.500,TSY,0.101,0.994,AAA,1,1,Same Day
2,2 Year Treasury,0.000,0.500,TSY,0.156,1.986,AAA,1,1,Same Day
3,3 Year Treasury,0.000,0.500,TSY,0.213,2.937,AAA,1,1,Same Day
4,5 Year Treasury,0.000,0.200,TSY,0.422,4.957,AAA,1,1,Same Day
5,7 Year Treasury,0.000,0.200,TSY,0.703,6.893,AAA,1,1,Same Day
6,10 Year Treasury,0.000,0.200,TSY,0.969,9.648,AAA,1,1,Same Day
7,30 Year Treasury,0.000,0.100,TSY,1.731,24.509,AAA,1,1,Same Day
8,2 Year ABS AAA,0.000,0.200,ABS,0.367,1.856,AAA,1,1,Same Day
9,5 Year ABS AAA,0.000,0.200,ABS,0.876,3.916,AAA,1,1,Same Day


## Standardizing Columns

In [35]:
print(df_assets.columns.tolist())

['Asset', 'Asset Level Min Weight', 'Asset Level Max Weight', 'Sector ', 'Yield', 'Duration', 'Quality', 'Liquidity Tier', 'Quality_Num', 'Liquidity_Label']


## Need to remove spaces inbetween words and strip leading/trailing spaces for easier readability

In [36]:
df_assets.columns = df_assets.columns.str.strip().str.lower().str.replace(' ', '_')   # <-- Removing leading/trailing spaces and removing spaces and substituting with underscores
print(df_assets.columns.tolist())

['asset', 'asset_level_min_weight', 'asset_level_max_weight', 'sector', 'yield', 'duration', 'quality', 'liquidity_tier', 'quality_num', 'liquidity_label']


Column names are clean now, easy to read.

## Let us now create the attributes for our optimization model

In [37]:
import cvxpy as cp
import numpy as np

# ---------- objective & attribute vectors ----------
y    = df_assets['yield'].values
dur  = df_assets['duration'].values
qnum = df_assets['quality_num'].values           # 1=AAA, 2=AA+, ...
liq  = df_assets['liquidity_label'].values       # 'Same Day', 'T+2', ...

wmin = df_assets['asset_level_min_weight'].values
wmax = df_assets['asset_level_max_weight'].values
sector = df_assets['sector'].values

n  = len(df_assets)
w  = cp.Variable(n)                              # decision weights


## Setting constraints

In [38]:
# ------------------------------------------------------------
# STEP 1 : assemble every rule as a CVXPY constraint
# ------------------------------------------------------------

constraints = [
# 1️  Asset-level bounds
    w >= wmin,
    w <= wmax,

# 2️  Fully invested (weights add to 100 %)
    cp.sum(w) == 1,

# 3️  Portfolio duration between 2 and 8 years
    dur @ w >= 2,
    dur @ w <= 8,

# 4️  Minimum average rating = A+  ⇒  numeric score ≤ 5
    qnum @ w <= 5,
]

# 5️  Same-day liquidity ≥ 20 %
same_day_mask = (liq == 'Same Day')          # Boolean vector
constraints.append(cp.sum(w[same_day_mask]) >= 0.20)

# 6️  Sector min / max weights
sector_bounds = {
    'TSY'        : (0.10, 1.00),
    'ABS'        : (0.00, 0.20),
    'MBS'        : (0.00, 0.40),
    'Corp'       : (0.00, 0.50),
    'High Yield' : (0.00, 0.05)
}

for sec, (lo, hi) in sector_bounds.items():
    mask = (sector == sec)
    if lo > 0:
        constraints.append(cp.sum(w[mask]) >= lo)
    constraints.append(cp.sum(w[mask]) <= hi)

print(f"{len(constraints)} constraints assembled.")


13 constraints assembled.


## Setting objective for our solver to optimize

# Our Base model ECOS from CVXPY

In [39]:
# ------------------------------------------------------------
# STEP 2 : Maximise portfolio yield subject to the constraints
# ------------------------------------------------------------

objective = cp.Maximize(y @ w)     # weighted-average portfolio yield

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.ECOS)         # ECOS solver in CVXPY

print("Solver status :", prob.status)
if prob.status not in ("optimal", "optimal_inaccurate"):
    raise ValueError("Problem infeasible or unbounded; review constraints.")

# ---------- attach weights back to the DataFrame --------------
df_assets['optimal_weight'] = np.round(w.value, 6)

# ---------- quick portfolio diagnostics -----------------------
port_yield      = float(y    @ w.value)
port_duration   = float(dur  @ w.value)
avg_rating_num  = float(qnum @ w.value)          # lower = better
same_day_share  = float(np.sum(w.value[same_day_mask]))

print(f"Portfolio yield     : {port_yield:.4f}")
print(f"Portfolio duration  : {port_duration:.2f}  yrs  (target 2–8)")
print(f"Average rating num  : {avg_rating_num:.2f}  (≤ 5  ⇒ ≥ A+)")
print(f"Same-day liquidity  : {same_day_share:.2%}  (≥ 20 %)")

# ---------- sector breakdown ----------------------------------
sector_summary = (
    df_assets.groupby('sector')['optimal_weight']
             .sum().sort_values(ascending=False)
             .to_frame('sector_weight')
)
display(sector_summary.style.format({'sector_weight': '{:.2%}'}))

# ---------- peek at first few positions -----------------------
display(df_assets[['asset', 'sector', 'yield',
                   'optimal_weight']].head())


Solver status : optimal
Portfolio yield     : 2.0363
Portfolio duration  : 7.86  yrs  (target 2–8)
Average rating num  : 5.00  (≤ 5  ⇒ ≥ A+)
Same-day liquidity  : 40.00%  (≥ 20 %)


,sector_weight
sector,
Corp,45.00%
MBS,25.00%
TSY,15.00%
ABS,10.00%
High Yield,5.00%


,asset,sector,yield,optimal_weight
0,6 Month Treasury,TSY,0.095,0.050
1,1 Year Treasury,TSY,0.101,-0.000
2,2 Year Treasury,TSY,0.156,-0.000
3,3 Year Treasury,TSY,0.213,-0.000
4,5 Year Treasury,TSY,0.422,0.000


In [40]:
# ─────────────────────────────────────────────────────────────
# 0. Imports
# ─────────────────────────────────────────────────────────────
import time
!pip install -q pulp ortools scipy


# ─────────────────────────────────────────────────────────────
# 1. Constraint-builder helpers  (needs to be *before* solvers)
# ─────────────────────────────────────────────────────────────
def build_cp_constraints(w):
    import cvxpy as cp
    cons = [w >= wmin, w <= wmax,
            cp.sum(w) == 1,
            dur @ w >= 2, dur @ w <= 8,
            qnum @ w <= 5,
            cp.sum(w[same_day_mask]) >= 0.20]
    for sec,(lo,hi) in sector_bounds.items():
        mask = (sector == sec)
        if lo > 0: cons.append(cp.sum(w[mask]) >= lo)
        cons.append(cp.sum(w[mask]) <= hi)
    return cons

def build_pulp_constraints(prob, w):
    import pulp
    prob += pulp.lpSum(w.values()) == 1
    for i in range(n):
        prob += w[i] >= wmin[i]; prob += w[i] <= wmax[i]
    prob += pulp.lpSum(w[i]*dur[i] for i in range(n)) >= 2
    prob += pulp.lpSum(w[i]*dur[i] for i in range(n)) <= 8
    prob += pulp.lpSum(w[i]*qnum[i] for i in range(n)) <= 5
    idx_sd = np.where(same_day_mask)[0]
    prob += pulp.lpSum(w[i] for i in idx_sd) >= 0.20
    for sec,(lo,hi) in sector_bounds.items():
        idx = np.where(sector==sec)[0]
        if lo>0: prob += pulp.lpSum(w[i] for i in idx) >= lo
        prob += pulp.lpSum(w[i] for i in idx) <= hi

def build_ort_constraints(solver, w):
    lin = lambda items: solver.Sum(v*c for v,c in items)
    solver.Add(lin([(w[i],1) for i in range(n)]) == 1)
    for i in range(n): solver.Add(w[i] >= wmin[i])
    solver.Add(lin([(w[i],dur[i]) for i in range(n)]) >= 2)
    solver.Add(lin([(w[i],dur[i]) for i in range(n)]) <= 8)
    solver.Add(lin([(w[i],qnum[i]) for i in range(n)]) <= 5)
    idx_sd = np.where(same_day_mask)[0]
    solver.Add(lin([(w[i],1) for i in idx_sd]) >= 0.20)
    for sec,(lo,hi) in sector_bounds.items():
        idx=np.where(sector==sec)[0]
        if lo>0: solver.Add(lin([(w[i],1) for i in idx]) >= lo)
        solver.Add(lin([(w[i],1) for i in idx]) <= hi)

# ─────────────────────────────────────────────────────────────
# 2. Solver wrappers
# ─────────────────────────────────────────────────────────────
def solve_cvxpy(chosen_solver='ECOS'):
    import cvxpy as cp
    w = cp.Variable(n)
    prob = cp.Problem(cp.Maximize(y @ w), build_cp_constraints(w))
    t0=time.time(); prob.solve(solver=chosen_solver, verbose=False); t1=time.time()
    return w.value, prob.value, t1-t0, prob.status

import pulp
def solve_pulp():
    prob = pulp.LpProblem("yieldMax", pulp.LpMaximize)
    w = pulp.LpVariable.dicts("w", range(n), lowBound=0)
    prob += pulp.lpSum(w[i]*y[i] for i in range(n))
    build_pulp_constraints(prob, w)
    t0=time.time(); prob.solve(pulp.PULP_CBC_CMD(msg=0)); t1=time.time()
    w_opt = np.array([pulp.value(w[i]) for i in range(n)])
    return w_opt, pulp.value(prob.objective), t1-t0, pulp.LpStatus[prob.status]

from ortools.linear_solver import pywraplp
def solve_ortools():
    s = pywraplp.Solver.CreateSolver('GLOP')
    w = [s.NumVar(0, wmax[i], f'w[{i}]') for i in range(n)]
    s.Maximize(s.Sum(w[i]*y[i] for i in range(n)))
    build_ort_constraints(s, w)
    t0=time.time(); s.Solve(); t1=time.time()
    w_opt = np.array([w[i].solution_value() for i in range(n)])
    return w_opt, s.Objective().Value(), t1-t0, 'optimal'

from scipy.optimize import minimize
def solve_slsqp():
    bounds = list(zip(wmin, wmax))
    def obj(w): return -(y @ w)
    cons = [{'type':'eq',   'fun': lambda w: np.sum(w)-1},
            {'type':'ineq', 'fun': lambda w:  dur @ w - 2},
            {'type':'ineq', 'fun': lambda w:  8 - dur @ w},
            {'type':'ineq', 'fun': lambda w:  5 - qnum @ w},
            {'type':'ineq', 'fun': lambda w:  np.sum(w[same_day_mask]) - 0.20}]
    for sec,(lo,hi) in sector_bounds.items():
        idx=np.where(sector==sec)[0]
        if lo>0: cons.append({'type':'ineq','fun':lambda w,idx=idx,lo=lo: np.sum(w[idx])-lo})
        cons.append({'type':'ineq','fun':lambda w,idx=idx,hi=hi: hi-np.sum(w[idx])})
    t0=time.time()
    res = minimize(obj, np.full(n,1/n), method='SLSQP', bounds=bounds,
                   constraints=cons, options={'disp':False})
    t1=time.time()
    return res.x, -res.fun, t1-t0, res.message

# ─────────────────────────────────────────────────────────────
# 3. Run all engines & collect metrics
# ─────────────────────────────────────────────────────────────
engines = [('CVXPY-ECOS', solve_cvxpy),
           ('PuLP-CBC',   solve_pulp),
           ('ORtools-GLOP', solve_ortools),
           ('SciPy-SLSQP',  solve_slsqp)]

weights_map, summary = {}, {}
for name, fn in engines:
    w_opt, obj, t, stat = fn()
    weights_map[name] = w_opt
    summary[name] = {'yield%':obj*100,
                     'duration':dur@w_opt,
                     'sameDay_%':np.sum(w_opt[same_day_mask]),
                     'sector_HY%':np.sum(w_opt[sector=='High Yield']),
                     'runtime (s)':t,
                     'status':stat}

# ── sector-split table
unique_sectors = sorted(np.unique(sector))
sector_rec = {name:{s:np.sum(w[sector==s]) for s in unique_sectors}
              for name,w in weights_map.items()}
tbl_sector = pd.DataFrame(sector_rec).T[unique_sectors]

# ── KPI table
kpi_rec = {n:{'yield':y@w,'duration':dur@w,'rating_num':qnum@w,
              'sameDay_pct':np.sum(w[same_day_mask])}
           for n,w in weights_map.items()}
tbl_kpi = pd.DataFrame(kpi_rec).T

# ─────────────────────────────────────────────────────────────
# 4. Display
# ─────────────────────────────────────────────────────────────
pd.options.display.float_format = '{:.3f}'.format
display(pd.DataFrame(summary).T)
display(tbl_sector.style.format('{:.1%}').set_caption('Sector allocation'))
display(tbl_kpi.style.format({'yield':'{:.4f}','duration':'{:.2f}',
                              'rating_num':'{:.2f}','sameDay_pct':'{:.1%}'}))


,yield%,duration,sameDay_%,sector_HY%,runtime (s),status
CVXPY-ECOS,203.628,7.860,0.400,0.050,0.026,optimal
PuLP-CBC,203.628,7.860,0.400,0.050,0.010,Optimal
ORtools-GLOP,203.628,7.860,0.400,0.050,0.001,optimal
SciPy-SLSQP,203.628,7.860,0.400,0.050,0.029,Optimization terminated successfully


,ABS,Corp,High Yield,MBS,TSY
CVXPY-ECOS,10.0%,45.0%,5.0%,25.0%,15.0%
PuLP-CBC,10.0%,45.0%,5.0%,25.0%,15.0%
ORtools-GLOP,10.0%,45.0%,5.0%,25.0%,15.0%
SciPy-SLSQP,10.0%,45.0%,5.0%,25.0%,15.0%


,yield,duration,rating_num,sameDay_pct
CVXPY-ECOS,2.0363,7.86,5.00,40.0%
PuLP-CBC,2.0363,7.86,5.00,40.0%
ORtools-GLOP,2.0363,7.86,5.00,40.0%
SciPy-SLSQP,2.0363,7.86,5.00,40.0%


In [41]:
# ═════════════════════════════════════════════════════════════════════
# 0. Installation
# ══════════════════════════════════════════════════════════════════════
!pip install -q cvxopt

# ────────────────────────────────────────────────────────────────────────
# 1.  Wrapper: solve_cvxpy() already defined earlier.
#     Here we add a thin helper that catches unsupported combos
#     (e.g. OSQP may complain if the objective is strictly linear)
# ────────────────────────────────────────────────────────────────────────
def try_cvxpy_solver(name):
    try:
        w_opt, obj, t, stat = solve_cvxpy(name)
    except Exception as e:                 # any failure → mark NA
        print(f"⚠️  {name} failed: {e}")
        w_opt  = np.full(n, np.nan)
        obj, t = np.nan, np.nan
        stat   = 'error'
    return w_opt, obj, t, stat

# ────────────────────────────────────────────────────────────────────────
# 2.  Run the four CVXPY solvers
# ────────────────────────────────────────────────────────────────────────
cvx_engines = ['ECOS', 'SCS', 'OSQP', 'CVXOPT']

weights_map, summary = {}, {}
for sname in cvx_engines:
    w_opt, obj, t, stat = try_cvxpy_solver(sname)
    weights_map[sname] = w_opt
    summary[sname] = {
        'yield%'   : obj * 100,
        'dur'      : dur @ w_opt,
        'sameDay%': np.nansum(w_opt[same_day_mask]),
        'secHY%':  np.nansum(w_opt[sector=='High Yield']),
        'time(s)'  : t,
        'status'   : stat
    }

# ────────────────────────────────────────────────────────────────────────
# 3.  Sector-split table
# ────────────────────────────────────────────────────────────────────────
unique_sectors = sorted(np.unique(sector))
sector_rec = {name: {sec: np.nansum(w[sector==sec])
                     for sec in unique_sectors}
              for name, w in weights_map.items()}
tbl_sector = (pd.DataFrame(sector_rec).T[unique_sectors]
              .round(3))

# ────────────────────────────────────────────────────────────────────────
# 4.  KPI table (yield, duration, rating, liquidity)
# ────────────────────────────────────────────────────────────────────────
kpi_rec = {name: {'yield'      : y @ w,
                  'duration'   : dur @ w,
                  'rating_num' : qnum @ w,
                  'sameDay_pct': np.nansum(w[same_day_mask])}
           for name, w in weights_map.items()}
tbl_kpi = (pd.DataFrame(kpi_rec).T
           .round({'yield':4, 'duration':2,
                   'rating_num':2, 'sameDay_pct':3}))

# ────────────────────────────────────────────────────────────────────────
# 5.  Display three tables
# ────────────────────────────────────────────────────────────────────────
pd.options.display.float_format = '{:.3f}'.format

print("◆ Runtime / objective comparison")
display(pd.DataFrame(summary).T
          .style.format({'yield%':'{:.3f}',
                         'dur':'{:.2f}',
                         'sameDay%':'{:.1%}',
                         'secHY%':'{:.1%}',
                         'time(s)':'{:.3f}'}))

print("\n◆ Sector allocation by solver")
display(tbl_sector.style.format('{:.1%}'))

print("\n◆ Portfolio KPIs")
display(tbl_kpi.style.format({
    'yield':'{:.4f}',
    'duration':'{:.2f}',
    'rating_num':'{:.2f}',
    'sameDay_pct':'{:.1%}'}))


◆ Runtime / objective comparison


,yield%,dur,sameDay%,secHY%,time(s),status
ECOS,203.628,7.86,40.0%,5.0%,0.016,optimal
SCS,203.628,7.86,40.0%,5.0%,0.023,optimal
OSQP,203.630,7.86,40.0%,5.0%,0.019,optimal
CVXOPT,203.628,7.86,40.0%,5.0%,0.023,optimal



◆ Sector allocation by solver


,ABS,Corp,High Yield,MBS,TSY
ECOS,10.0%,45.0%,5.0%,25.0%,15.0%
SCS,10.0%,45.0%,5.0%,25.0%,15.0%
OSQP,10.0%,45.0%,5.0%,25.0%,15.0%
CVXOPT,10.0%,45.0%,5.0%,25.0%,15.0%



◆ Portfolio KPIs


,yield,duration,rating_num,sameDay_pct
ECOS,2.0363,7.86,5.00,40.0%
SCS,2.0363,7.86,5.00,40.0%
OSQP,2.0363,7.86,5.00,40.0%
CVXOPT,2.0363,7.86,5.00,40.0%


## Eval, benchmarking

In [42]:
# -----------------------------------------------------------------------------
# Extra evaluation metrics
# -----------------------------------------------------------------------------

def max_abs_violation(w):
    viol = []
    viol.append(abs(np.sum(w)-1))                         # weights sum
    viol.append(max(0, 2 - dur@w, dur@w - 8))             # duration band
    viol.append(max(0, qnum@w - 5))                       # rating
    viol.append(max(0, 0.20 - np.sum(w[same_day_mask])))  # liquidity
    for sec,(lo,hi) in sector_bounds.items():
        sw = np.sum(w[sector==sec])
        viol.append(max(0, lo - sw, sw - hi))
    return max(viol)

for name, w in weights_map.items():
    # worst constraint violation
    summary[name]['max_violation'] = max_abs_violation(w)

# Objective gap vs ECOS benchmark
obj_ref = summary['ECOS']['yield%']
for name in summary:
    summary[name]['gap_bp'] = (summary[name]['yield%'] - obj_ref)*100

# Pretty print with new columns
cols = ['yield%','gap_bp','dur','max_violation','time(s)','status']
display(pd.DataFrame(summary).T[cols].round({
    'yield%':3,'gap_bp':2,'dur':2,
    'sameDay%':3,'secHY%':3,'time(s)':3,
    'max_violation':2}))


,yield%,gap_bp,dur,max_violation,time(s),status
ECOS,203.628,0.000,7.860,0.000,0.016,optimal
SCS,203.628,0.000,7.860,0.000,0.023,optimal
OSQP,203.630,0.252,7.860,0.000,0.019,optimal
CVXOPT,203.628,0.000,7.860,0.000,0.023,optimal


##Monte_carlo perturbation

In [25]:
# ╔═════════════════════════════════════════════════════════════════════╗
#   Sensitivity test: 100 random perturbations of the input data
# ╚═════════════════════════════════════════════════════════════════════╝
N_SIM      = 100
SIGMA_YLD  = 0.0001      # 1-bp σ noise on yield
SIGMA_DUR  = 0.05        # 0.05-yr σ noise on duration
SIGMA_RTG  = 0.1         # 0.1 notch noise on numeric rating

base_w, base_obj, *_ = solve_cvxpy('ECOS')        # baseline (no noise)

def run_perturb(idx):
    y_p   = y    + np.random.normal(0, SIGMA_YLD, n)
    dur_p = dur  + np.random.normal(0, SIGMA_DUR, n)
    q_p   = qnum + np.random.normal(0, SIGMA_RTG, n)

    # rebuild constraints with perturbed vectors
    def build_cp_pert(w):
        import cvxpy as cp
        cons = [w >= wmin, w <= wmax,
                cp.sum(w) == 1,
                dur_p @ w >= 2, dur_p @ w <= 8,
                q_p   @ w <= 5,
                cp.sum(w[same_day_mask]) >= 0.20]
        for sec,(lo,hi) in sector_bounds.items():
            mask = (sector == sec)
            if lo>0: cons.append(cp.sum(w[mask]) >= lo)
            cons.append(cp.sum(w[mask]) <= hi)
        return cons

    import cvxpy as cp
    w = cp.Variable(n)
    prob = cp.Problem(cp.Maximize(y_p @ w), build_cp_pert(w))
    prob.solve(solver='ECOS', verbose=False)
    return (w.value,
            y_p @ w.value,
            dur_p @ w.value,
            np.sum(w.value[same_day_mask]))

# run sims
sim_res = [run_perturb(i) for i in range(N_SIM)]

# gather dispersion metrics
sim_obj   = np.array([r[1] for r in sim_res])
sim_dur   = np.array([r[2] for r in sim_res])
sim_liq   = np.array([r[3] for r in sim_res])
sim_delta = np.array([np.linalg.norm(r[0]-base_w,1) for r in sim_res])  # L1 diff

print(f"Objective  σ: {sim_obj.std()*100:.4f} bp")
print(f"Duration   σ: {sim_dur.std():.4f} yrs")
print(f"Liquidity  σ: {sim_liq.std()*100:.2f}%")
print(f"Median L1 weight shift: {np.median(sim_delta)*100:.2f}%")


Objective  σ: 0.2343 bp
Duration   σ: 0.0000 yrs
Liquidity  σ: 2.37%
Median L1 weight shift: 25.09%


In [43]:
# ╔══════════════════════════════════════════════════════════════════╗
# 0. One-time install of memory_profiler + psutil
# ╚══════════════════════════════════════════════════════════════════╝
!pip install -q memory_profiler psutil

from memory_profiler import memory_usage
import gc, warnings; warnings.filterwarnings('ignore')

# helper: run fn() and capture peak MiB above baseline
def run_with_mem(fn):
    gc.collect()                       # clean slate
    base = memory_usage(-1, interval=.1, timeout=1)[0]    # current MB
    mem, ret = memory_usage((fn, ()), retval=True, max_usage=True, interval=.05)
    peak = mem - base
    return peak, ret                  # MB, (w,obj,time,status)

# ─────────────────────────────────────────────────────────────
# 1. Build dict of engines (reuse your wrappers)
# ─────────────────────────────────────────────────────────────
engines = {
    'ECOS'        : lambda : solve_cvxpy('ECOS'),
    'SCS'         : lambda : solve_cvxpy('SCS'),
    'OSQP'        : lambda : solve_cvxpy('OSQP'),
    'CVXOPT'      : lambda : solve_cvxpy('CVXOPT'),
    'PuLP-CBC'    : solve_pulp,
    'ORtools-GLOP': solve_ortools,
    'SciPy-SLSQP' : solve_slsqp
}

# ─────────────────────────────────────────────────────────────
# 2. Run + collect CPU time and peak MiB
# ─────────────────────────────────────────────────────────────
mem_tbl, weights_map = {}, {}
for name, fn in engines.items():
    peak_mb, (w,obj,t,stat) = run_with_mem(fn)
    weights_map[name] = w
    mem_tbl[name] = {'peak_MB': peak_mb,
                     'time_s' : t,
                     'yield_bp': obj*100,
                     'status' : stat}

# ─────────────────────────────────────────────────────────────
# 3. Display
# ─────────────────────────────────────────────────────────────
pd.options.display.float_format = '{:.3f}'.format
display(pd.DataFrame(mem_tbl).T
        .sort_values('peak_MB')
        .style.format({'peak_MB':'{:.1f}', 'time_s':'{:.3f}',
                       'yield_bp':'{:.3f}'}))


,peak_MB,time_s,yield_bp,status
SCS,0.0,0.028,203.628,optimal
OSQP,0.0,0.030,203.630,optimal
CVXOPT,0.0,0.033,203.628,optimal
PuLP-CBC,0.0,0.009,203.628,Optimal
ORtools-GLOP,0.0,0.001,203.628,optimal
SciPy-SLSQP,0.0,0.040,203.628,Optimization terminated successfully
ECOS,0.0,0.040,203.628,optimal


## Appendix Experimenting!

In [31]:
# ------------------------------------------------------------
# STEP 2 : maximise portfolio yield subject to the constraints with SCS solver
# ------------------------------------------------------------

objective = cp.Maximize(y @ w)     # weighted-average portfolio yield

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.SCS)         # SCS is bundled with CVXPY

print("Solver status :", prob.status)
if prob.status not in ("optimal", "optimal_inaccurate"):
    raise ValueError("Problem infeasible or unbounded; review constraints.")

# ---------- attach weights back to the DataFrame --------------
df_assets['optimal_weight'] = np.round(w.value, 6)

# ---------- quick portfolio diagnostics -----------------------
port_yield      = float(y    @ w.value)
port_duration   = float(dur  @ w.value)
avg_rating_num  = float(qnum @ w.value)          # lower = better
same_day_share  = float(np.sum(w.value[same_day_mask]))

print(f"Portfolio yield     : {port_yield:.4f}")
print(f"Portfolio duration  : {port_duration:.2f}  yrs  (target 2–8)")
print(f"Average rating num  : {avg_rating_num:.2f}  (≤ 5  ⇒ ≥ A+)")
print(f"Same-day liquidity  : {same_day_share:.2%}  (≥ 20 %)")

# ---------- sector breakdown ----------------------------------
sector_summary = (
    df_assets.groupby('sector')['optimal_weight']
             .sum().sort_values(ascending=False)
             .to_frame('sector_weight')
)
display(sector_summary.style.format({'sector_weight': '{:.2%}'}))

# ---------- peek at first few positions -----------------------
display(df_assets[['asset', 'sector', 'yield',
                   'optimal_weight']].head())


Solver status : optimal
Portfolio yield     : 2.0363
Portfolio duration  : 7.86  yrs  (target 2–8)
Average rating num  : 5.00  (≤ 5  ⇒ ≥ A+)
Same-day liquidity  : 40.00%  (≥ 20 %)


,sector_weight
sector,
Corp,45.00%
MBS,25.00%
TSY,15.00%
ABS,10.00%
High Yield,5.00%


,asset,sector,yield,optimal_weight
0,6 Month Treasury,TSY,0.095,0.05
1,1 Year Treasury,TSY,0.101,-0.00
2,2 Year Treasury,TSY,0.156,-0.00
3,3 Year Treasury,TSY,0.213,-0.00
4,5 Year Treasury,TSY,0.422,0.00


In [32]:
# ------------------------------------------------------------
# STEP 2 : maximise portfolio yield subject to the constraints with OSQP
# ------------------------------------------------------------

objective = cp.Maximize(y @ w)     # weighted-average portfolio yield

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.OSQP)         # OSQP is bundled with CVXPY

print("Solver status :", prob.status)
if prob.status not in ("optimal", "optimal_inaccurate"):
    raise ValueError("Problem infeasible or unbounded; review constraints.")

# ---------- attach weights back to the DataFrame --------------
df_assets['optimal_weight'] = np.round(w.value, 6)

# ---------- quick portfolio diagnostics -----------------------
port_yield      = float(y    @ w.value)
port_duration   = float(dur  @ w.value)
avg_rating_num  = float(qnum @ w.value)          # lower = better
same_day_share  = float(np.sum(w.value[same_day_mask]))

print(f"Portfolio yield     : {port_yield:.4f}")
print(f"Portfolio duration  : {port_duration:.2f}  yrs  (target 2–8)")
print(f"Average rating num  : {avg_rating_num:.2f}  (≤ 5  ⇒ ≥ A+)")
print(f"Same-day liquidity  : {same_day_share:.2%}  (≥ 20 %)")

# ---------- sector breakdown ----------------------------------
sector_summary = (
    df_assets.groupby('sector')['optimal_weight']
             .sum().sort_values(ascending=False)
             .to_frame('sector_weight')
)
display(sector_summary.style.format({'sector_weight': '{:.2%}'}))

# ---------- peek at first few positions -----------------------
display(df_assets[['asset', 'sector', 'yield',
                   'optimal_weight']].head())


Solver status : optimal
Portfolio yield     : 2.0363
Portfolio duration  : 7.86  yrs  (target 2–8)
Average rating num  : 5.00  (≤ 5  ⇒ ≥ A+)
Same-day liquidity  : 40.00%  (≥ 20 %)


,sector_weight
sector,
Corp,45.00%
MBS,25.00%
TSY,15.00%
ABS,10.00%
High Yield,5.00%


,asset,sector,yield,optimal_weight
0,6 Month Treasury,TSY,0.095,0.049996
1,1 Year Treasury,TSY,0.101,-0.000004
2,2 Year Treasury,TSY,0.156,-0.000003
3,3 Year Treasury,TSY,0.213,-0.000003
4,5 Year Treasury,TSY,0.422,-0.000002


In [33]:
# ------------------------------------------------------------
# STEP 2 : maximise portfolio yield subject to the constraints with CVXOPT
# ------------------------------------------------------------

objective = cp.Maximize(y @ w)     # weighted-average portfolio yield

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.CVXOPT)         # CVXOPT is bundled with CVXPY

print("Solver status :", prob.status)
if prob.status not in ("optimal", "optimal_inaccurate"):
    raise ValueError("Problem infeasible or unbounded; review constraints.")

# ---------- attach weights back to the DataFrame --------------
df_assets['optimal_weight'] = np.round(w.value, 6)

# ---------- quick portfolio diagnostics -----------------------
port_yield      = float(y    @ w.value)
port_duration   = float(dur  @ w.value)
avg_rating_num  = float(qnum @ w.value)          # lower = better
same_day_share  = float(np.sum(w.value[same_day_mask]))

print(f"Portfolio yield     : {port_yield:.4f}")
print(f"Portfolio duration  : {port_duration:.2f}  yrs  (target 2–8)")
print(f"Average rating num  : {avg_rating_num:.2f}  (≤ 5  ⇒ ≥ A+)")
print(f"Same-day liquidity  : {same_day_share:.2%}  (≥ 20 %)")

# ---------- sector breakdown ----------------------------------
sector_summary = (
    df_assets.groupby('sector')['optimal_weight']
             .sum().sort_values(ascending=False)
             .to_frame('sector_weight')
)
display(sector_summary.style.format({'sector_weight': '{:.2%}'}))

# ---------- peek at first few positions -----------------------
display(df_assets[['asset', 'sector', 'yield',
                   'optimal_weight']].head())


Solver status : optimal
Portfolio yield     : 2.0363
Portfolio duration  : 7.86  yrs  (target 2–8)
Average rating num  : 5.00  (≤ 5  ⇒ ≥ A+)
Same-day liquidity  : 40.00%  (≥ 20 %)


,sector_weight
sector,
Corp,45.00%
MBS,25.00%
TSY,15.00%
ABS,10.00%
High Yield,5.00%


,asset,sector,yield,optimal_weight
0,6 Month Treasury,TSY,0.095,0.05
1,1 Year Treasury,TSY,0.101,-0.00
2,2 Year Treasury,TSY,0.156,-0.00
3,3 Year Treasury,TSY,0.213,-0.00
4,5 Year Treasury,TSY,0.422,0.00
